In [1]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import string
import re
import os

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
from simpletransformers.t5 import T5Model, T5Args

import re
import pandas as pd
from nltk import tokenize

import numpy as np
import string
from tqdm import tqdm_notebook
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/andrey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


комментарии текста ниже есть в main_tf_idf.py

In [2]:
stop_words = pd.read_csv('stopwords.txt')
stop_words = list(stop_words['c'])
stop_words = stop_words + ['URL','USER','HASHTAG','ap','ар', '2ар', 'interfax', 'ru','ak']

In [3]:
def removeStopwords(wordlist, stopwords):
    return [w for w in (wordlist) if w not in stopwords]

translator = str.maketrans('', '', string.punctuation)

In [4]:
def preprocess_text(Text,punct=False, figures=False, stopwords = False):
    result=[]
    for sentense in tqdm_notebook(Text):
        string=(sentense.lower())
        string = [word for word in string.split() if word.startswith('@') == False]
        string = ' '.join([word for word in string if word.startswith('http') == False])
        
        string = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', ' ', string)
        string = re.sub('([>:;=-]+[\(cсCС/\\[L{<@]+)|([\(]{2,})', ' ', string)
        # # заменяем веселые/радостные смайлики на общий токен
        string = re.sub('([>:;=-]+[\)dD*pPрРbB]+)|([\)]{2,})', ' ', string)

    
        string = re.sub('[^a-zA-Zа-яА-Я0-9]+', ' ', string)
        
        
        
        if punct==True:
            string=string.translate(translator)
        if figures==True:
            string=re.sub(r'\d+', '', string)
            
        clear_sentence = string.split()
        if stopwords == True:
            clear_sentence = removeStopwords(clear_sentence,stop_words)
        
        
        
        
        clear_sentence = ' '.join(clear_sentence)
        

        
        result.append(clear_sentence)
    return result

In [5]:
def dataset_creation(data:list):
    df_val = pd.DataFrame()

    test_count = []

    for num_story in tqdm_notebook(range(len(data))):

        story  = data[num_story]
        test_count.append(len(story['news']))
        for num_news, elem in enumerate(story['news']):
            try:
                del elem['codes']
            except:
                  pass

            try:
                df1 = pd.DataFrame(elem, index=[0])
                df1['mega_title'] = story['title']
                df1['num_story'] = num_story
                df1['count'] = len(story['news'])
                df_val = df_val.append(df1,ignore_index=True)
            except:
                print(f'ERROR: num_story = {num_story} | num_news = {num_news} ')

    df_val.columns = ['id', 'lastUpd', 'type_id', 'language_id', 'slugline', 'title',
       'subheadline', 'trashline', 'dateline', 'text', 'background',
       'mega_title', 'num_story', 'count']

    return df_val

In [6]:
os.listdir('data_interfax/')

['dataset_public.json', 'codes_reference.csv']

### main

In [7]:
file_name = 'data_interfax/dataset_public.json'

In [8]:
with open(file_name,'r') as file:
    data = json.load(file)

assert type(data) == list ,'Должны быть списки сюжетов'

dataset = dataset_creation(data)

/home/andrey/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511 entries, 0 to 1510
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1511 non-null   int64 
 1   lastUpd      1511 non-null   object
 2   type_id      1511 non-null   object
 3   language_id  1511 non-null   object
 4   slugline     1511 non-null   object
 5   title        1424 non-null   object
 6   subheadline  0 non-null      object
 7   trashline    0 non-null      object
 8   dateline     20 non-null     object
 9   text         1511 non-null   object
 10  background   0 non-null      object
 11  mega_title   1511 non-null   object
 12  num_story    1511 non-null   int64 
 13  count        1511 non-null   int64 
dtypes: int64(3), object(11)
memory usage: 165.4+ KB


используем текст новостей "молний", как заголовки (не более 1 предложения)

In [10]:
dataset.loc[dataset[dataset['type_id'] == 'FLASH'].index,'title'] = dataset[dataset['type_id'] == 'FLASH']['text']

In [11]:
dataset.head().T

,0,1,2,3,4
id,26734410,26735670,26735344,26733711,26733691
lastUpd,2021-03-29 08:29:31.400063+00:00,2021-03-29 07:55:26.530522+00:00,2021-03-29 06:51:10.523786+00:00,2021-03-29 06:34:32.342461+00:00,2021-03-29 06:32:21.804681+00:00
type_id,NEWS,NEWS,NEWS,NEWS,EXPRESS
language_id,RU,RU,RU,RU,RU
slugline,РОССИЯ-ВАКЦИНАЦИЯ-ПРОТИВОПОКАЗАНИЯ,РОССИЯ-ВАКЦИНАЦИЯ-ПРОТИВОПОКАЗАНИЯ,РОССИЯ-ВАКЦИНА-ПРОИЗВОДСТВО,РОССИЯ-ВАКЦИНА-ПРОИЗВОДСТВО-2,ЭКСПРЕСС-РОССИЯ-ВАКЦИНА-ПРОИЗВОДСТВО
title,"Онкология, диабет и инфаркт не являются против...","Ар+2 Онкология, диабет и инфаркт не являются п...","Производство ""ЭпиВакКороны"" с апреля масштабир...","2Ар+1 Производство ""ЭпиВакКороны"" с апреля мас...","Ар++ Производство ""ЭпиВакКороны"" с апреля масш..."
subheadline,None,None,None,None,None
trashline,None,None,None,None,None
dateline,None,None,None,None,None
text,Москва. 29 марта. ИНТЕРФАКС - Хроническим боль...,Москва. 29 марта. ИНТЕРФАКС - Хроническим боль...,*** Очередная партия вакцины выйдет в гражданс...,(расширено после 2 абзаца)\n\nМосква. 29 марта...,Москва. 29 марта. ИНТЕРФАКС - Производство вак...


In [12]:
check = dataset.groupby(['mega_title'])['num_story'].nunique().reset_index()
check.sort_values(['num_story'],ascending=False)

,mega_title,num_story
53,Заражения COVID-19,6
19,Вакцина от COVID-19,5
79,Курсы валют,5
150,Торги фьючерсными контрактами,4
59,Значения индексов Мосбиржи и РТС,4
...,...,...
54,Заседание ЕАЭС,1
55,Заявления Байдена,1
56,Землетрясение в Монголии,1
57,Землетрясения,1


In [13]:
dataset = dataset[['title','mega_title','text','num_story']]
dataset = dataset.rename(columns={'title':'input_text','mega_title':'target_text','text':'text_all'})
dataset['input_text'] = preprocess_text(dataset['input_text'], True, stopwords=True)

dataset['target_text_prep'] = preprocess_text(dataset['target_text'], True, stopwords=False)

dataset['text_all'] = preprocess_text(dataset['text_all'], True, stopwords=True)
# убираем инфу до интерфакс (заглушка)
dataset['text_all'] = dataset['text_all'].apply(lambda x: x.split('интерфакс',1)[1] if 'интерфакс' in x else x)

# для МТ5
dataset['prefix'] = ['summarization']*len(dataset)

/home/andrey/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
def merge_text(dataset:pd.DataFrame):
    
    stories,story_raw,input_text,input_text_body,raw_title,target_text = [],[],[],[],[],[]

    for story in tqdm_notebook(set(dataset['num_story'])):

        subset = dataset[dataset['num_story']==story]

        cols = ['input_text','text_all','target_text']

        story_raw.append(list(subset['text_all'])[0])

        stories.append(story)
        
        target_text.append(subset['target_text'].unique()[0])
        for col in cols:
            subset_set = '.'.join(list(subset[col]))

            if col == 'input_text':
                input_text.append(subset_set)

            elif col == 'text_all':
                input_text_body.append(subset_set)




    df_all = pd.DataFrame([target_text,story_raw,input_text,input_text_body,stories] ).T
    df_all.columns = ['target_text','raw_target_text','input_text','text_all','num_story']
    df_all['prefix'] = ['summarization']*len(df_all)
    df_all= df_all[['prefix','input_text','text_all','target_text','num_story']]
    
    
    return df_all

In [15]:
df_all = merge_text(dataset)

/home/andrey/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [16]:
df_all

,prefix,input_text,text_all,target_text,num_story
0,summarization,онкология диабет инфаркт являются противопоказ...,хроническим больным числе онкологией инфаркта...,"Вакцина НИИ ""Вектор""",0
1,summarization,президент молдавии 26 марта проведет депутатам...,президент молдавии майя санду проведет пятниц...,Политический кризис в Молдавии,1
2,summarization,spacex вывела орбиту 60 интернет путников star...,тяжелая ракета носитель falcon 9 среду вывела...,Запуск спутников SpaceX,2
3,summarization,значения основных фондовых индексов стран амер...,24 03 2021 наименование значение предыдущ макс...,Значения фондовых индексов,3
4,summarization,рынок акций рф вечеру сохраняет консолидацию о...,рынок акций рф вечеру среду сохраняет консоли...,Рынок акций РФ,4
...,...,...,...,...,...
195,summarization,итоги торгов акциями компаний тэк основном рын...,01 02 2021 эмит цены сделок рублей тренд объем...,Результаты торгов Московской биржи,195
196,summarization,домодедово размещает 7 летние евробонды 453 мл...,домодедово размещает 7 летние евробонды 453 мл...,"Доходность евробондов ""Домодедово""",196
197,summarization,глава дипломатии ес намерен визита москву обсу...,европейская внешнеполитическая служба евс рас...,Визит Борреля в Москву,197
198,summarization,азербайджане растет суточный показатель зараже...,азербайджан медики азербайджане сутки выявили...,Заражения COVID-19,198


In [17]:
check = df_all.groupby(['target_text'])['num_story'].nunique().reset_index()
check.sort_values(['target_text'],ascending=False)

,target_text,num_story
163,Юнипро,1
162,Южного газовый коридора,1
161,Экспортные контракты на зерно,1
160,Экспорт из РФ,1
159,Шпионаже в пользу России в Болгарии,1
...,...,...
4,IPO Fix Price,1
3,COVID-19 в Молдавии,1
2,Замедление работы Twitter,1
1,Взрыв газа во Владикавказе,1


In [18]:
df_all.iloc[-1]['input_text']

'ответственность перевозчика участника крупного дтп самарой застрахована росгосстрахе.шестеро пострадавших дтп самарой подключены ивл.власти ульяновской самарской областей заявили выплате семьям погибших дтп самарой 1 млн рублей.губернатор самарской области объявил 31 января днем траура.пострадавших аварии автобусом самарой включая ребенка находятся тяжелом состоянии.траур погибшим дтп сызранью объявлен ульяновской области.пострадавших аварии автобусом самарой включая полугодовалого ребенка находятся реанимации'

In [19]:
df_all.iloc[0]['input_text']

'онкология диабет инфаркт являются противопоказанием вакцинации covid 19 глава екатеринбургского нии вектор.2 онкология диабет инфаркт являются противопоказанием вакцинации covid 19 глава екатеринбургского нии вектор.производство эпиваккороны апреля масштабируется 5 млн доз месяц глава екатеринбургского нии вектор.1 производство эпиваккороны апреля масштабируется 5 млн доз месяц глава екатеринбургского нии вектор.производство эпиваккороны апреля масштабируется 5 млн доз месяц глава екатеринбургского нии вектор'

In [20]:
dataset_old = dataset.copy()

In [21]:
dataset_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511 entries, 0 to 1510
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   input_text        1511 non-null   object
 1   target_text       1511 non-null   object
 2   text_all          1511 non-null   object
 3   num_story         1511 non-null   int64 
 4   target_text_prep  1511 non-null   object
 5   prefix            1511 non-null   object
dtypes: int64(1), object(5)
memory usage: 71.0+ KB


In [22]:
df_all

,prefix,input_text,text_all,target_text,num_story
0,summarization,онкология диабет инфаркт являются противопоказ...,хроническим больным числе онкологией инфаркта...,"Вакцина НИИ ""Вектор""",0
1,summarization,президент молдавии 26 марта проведет депутатам...,президент молдавии майя санду проведет пятниц...,Политический кризис в Молдавии,1
2,summarization,spacex вывела орбиту 60 интернет путников star...,тяжелая ракета носитель falcon 9 среду вывела...,Запуск спутников SpaceX,2
3,summarization,значения основных фондовых индексов стран амер...,24 03 2021 наименование значение предыдущ макс...,Значения фондовых индексов,3
4,summarization,рынок акций рф вечеру сохраняет консолидацию о...,рынок акций рф вечеру среду сохраняет консоли...,Рынок акций РФ,4
...,...,...,...,...,...
195,summarization,итоги торгов акциями компаний тэк основном рын...,01 02 2021 эмит цены сделок рублей тренд объем...,Результаты торгов Московской биржи,195
196,summarization,домодедово размещает 7 летние евробонды 453 мл...,домодедово размещает 7 летние евробонды 453 мл...,"Доходность евробондов ""Домодедово""",196
197,summarization,глава дипломатии ес намерен визита москву обсу...,европейская внешнеполитическая служба евс рас...,Визит Борреля в Москву,197
198,summarization,азербайджане растет суточный показатель зараже...,азербайджан медики азербайджане сутки выявили...,Заражения COVID-19,198


In [23]:
dataset = df_all[['prefix','input_text','target_text']]

In [24]:
dataset

,prefix,input_text,target_text
0,summarization,онкология диабет инфаркт являются противопоказ...,"Вакцина НИИ ""Вектор"""
1,summarization,президент молдавии 26 марта проведет депутатам...,Политический кризис в Молдавии
2,summarization,spacex вывела орбиту 60 интернет путников star...,Запуск спутников SpaceX
3,summarization,значения основных фондовых индексов стран амер...,Значения фондовых индексов
4,summarization,рынок акций рф вечеру сохраняет консолидацию о...,Рынок акций РФ
...,...,...,...
195,summarization,итоги торгов акциями компаний тэк основном рын...,Результаты торгов Московской биржи
196,summarization,домодедово размещает 7 летние евробонды 453 мл...,"Доходность евробондов ""Домодедово"""
197,summarization,глава дипломатии ес намерен визита москву обсу...,Визит Борреля в Москву
198,summarization,азербайджане растет суточный показатель зараже...,Заражения COVID-19


Load & evaluate model

In [25]:
model_args = T5Args()
model_args.max_seq_length = 300
model_args.train_batch_size = 2
model_args.eval_batch_size = 2
model_args.num_train_epochs = 10

model_args.evaluate_during_training = True #True
# model_args.evaluate_during_training_steps = 15000
model_args.evaluate_during_training_verbose = True

model_args.output_dir = 'fin_output1'

model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1

model_args.save_eval_checkpoints = True
model_args.save_model_every_epoch = False

model_args.no_cache = True

model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.num_return_sequences = 1

model_args.wandb_project = "summarization"


In [26]:
model_load = T5Model("mt5", "final/fin_output_Ann", args=model_args, use_cuda=False)

In [27]:
to_predict_train = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(dataset["prefix"].tolist(), dataset["input_text"].tolist())
]

In [28]:
preds_train = model_load.predict(to_predict_train)

/home/andrey/miniconda3/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3351: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


### оценка для нас

In [29]:
dataset['preds'] = preds_train

/home/andrey/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
dataset

,prefix,input_text,target_text,preds
0,summarization,онкология диабет инфаркт являются противопоказ...,"Вакцина НИИ ""Вектор""","Вакцина НИИ ""Вектор"""
1,summarization,президент молдавии 26 марта проведет депутатам...,Политический кризис в Молдавии,Новое правительство
2,summarization,spacex вывела орбиту 60 интернет путников star...,Запуск спутников SpaceX,Starlink от SpaceX от SpaceX от SpaceX от Spac...
3,summarization,значения основных фондовых индексов стран амер...,Значения фондовых индексов,Основные фондовые индексы
4,summarization,рынок акций рф вечеру сохраняет консолидацию о...,Рынок акций РФ,Курс акций РФ
...,...,...,...,...
195,summarization,итоги торгов акциями компаний тэк основном рын...,Результаты торгов Московской биржи,Результаты торгов Московской биржи
196,summarization,домодедово размещает 7 летние евробонды 453 мл...,"Доходность евробондов ""Домодедово""","Доходность евробондов ""Домодово"""
197,summarization,глава дипломатии ес намерен визита москву обсу...,Визит Борреля в Москву,Внутриполитический кризис в Москве
198,summarization,азербайджане растет суточный показатель зараже...,Заражения COVID-19,Заражений COVID-19


In [31]:
mega_titles = dataset['target_text'].unique()

In [32]:
df_preds = pd.DataFrame({'origin':dataset['target_text'].values,
                         'preds_train':preds_train})

In [33]:
df_preds

,origin,preds_train
0,"Вакцина НИИ ""Вектор""","Вакцина НИИ ""Вектор"""
1,Политический кризис в Молдавии,Новое правительство
2,Запуск спутников SpaceX,Starlink от SpaceX от SpaceX от SpaceX от Spac...
3,Значения фондовых индексов,Основные фондовые индексы
4,Рынок акций РФ,Курс акций РФ
...,...,...
195,Результаты торгов Московской биржи,Результаты торгов Московской биржи
196,"Доходность евробондов ""Домодедово""","Доходность евробондов ""Домодово"""
197,Визит Борреля в Москву,Внутриполитический кризис в Москве
198,Заражения COVID-19,Заражений COVID-19


Код оценки взят и переделан для наших нужд из https://colab.research.google.com/drive/1B26oDFEKSNCcI0BPkGXgxi13pbadriyN

In [34]:
# https://github.com/IlyaGusev/summarus/blob/master/summarus/util/metrics.py
from summarus.util.metrics import print_metrics 
# https://github.com/IlyaGusev/summarus/blob/master/summarus/evaluate.py
from evaluate import postprocess

def calc_method_score(references, predictions, nrows=None, verbose=True):
    for i, (ref, hyp) in enumerate(zip(references, predictions)):
        references[i], predictions[i] = postprocess(ref, hyp, language="ru", tokenize_after=True, lower=True)
    metrics = print_metrics(references, predictions, language="ru", meteor_jar="meteor-1.5/meteor-1.5.jar", verbose=verbose)
    return metrics

In [35]:
name_preds = 'preds_train'
df_preds['full_metrics'] = df_preds.apply(lambda x: calc_method_score([x['origin']], [x[name_preds]], nrows=None, verbose=False), axis=1)

df_preds['bleu'] = df_preds['full_metrics'].apply(lambda x: round(x["bleu"] * 100.0,3))
df_preds['rouge-1f'] = df_preds['full_metrics'].apply(lambda x: round(x["rouge-1"]['f'] * 100.0,3))
df_preds['rouge-2f'] = df_preds['full_metrics'].apply(lambda x: round(x["rouge-2"]['f'] * 100.0,3))
df_preds['rouge-lf'] = df_preds['full_metrics'].apply(lambda x: round(x["rouge-l"]['f'] * 100.0,3))

/home/andrey/miniconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/andrey/miniconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/andrey/miniconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use S

Здесь результаты работы на всем датасете предоставленном от интерфакса (на его части модель обучалась)

In [36]:
df_res = pd.DataFrame(dict(df_preds.mean(axis=0)), index=[0])
df_res['name'] = name_preds
df_res

,bleu,rouge-1f,rouge-2f,rouge-lf,name
0,59.397995,60.96968,45.40087,59.42367,preds_train
